In [1]:
%load_ext autoreload
%autoreload 2

In [49]:
import pandas as pd
import numpy as np
from fflpred_backend.model import main

In [8]:
%cd ..

/Users/armin/code/1kruidenp/fflpred_backend


In [9]:
!pwd

/Users/armin/code/1kruidenp/fflpred_backend


In [139]:
    #prediction = main()
    team = pd.DataFrame({
        'name': [
            'Harry kane',
            'Neal maupay',
            'heung-min son',
            'adam lallana',
            'stuart Dallas',
            'matt Targett',
            'Hugo lloris'
        ],
        'position': [
            'FWD',
            'FWD',
            'MID',
            'MID',
            'DEF',
            'DEF',
            'GK'
        ]
    })
    
    budget = 4.2

In [140]:
team_df = team.copy()
team_df['name'] = team_df.apply(lambda row: row['name'].lower(), axis = 1)

In [141]:
prediction[prediction['name']=='matt target']

,name,GW,season,value,predicted_points,position


In [142]:
team_df

,name,position
0,harry kane,FWD
1,neal maupay,FWD
2,heung-min son,MID
3,adam lallana,MID
4,stuart dallas,DEF
5,matt targett,DEF
6,hugo lloris,GK


In [143]:
team_df['predicted_points'] = team_df.apply(lambda row: prediction.predicted_points[prediction['name'] == row['name']].item(), axis = 1)

In [144]:
team_df

,name,position,predicted_points
0,harry kane,FWD,4.431238
1,neal maupay,FWD,0.665884
2,heung-min son,MID,4.290511
3,adam lallana,MID,0.993300
4,stuart dallas,DEF,4.681366
5,matt targett,DEF,2.859214
6,hugo lloris,GK,2.600000


In [150]:
prediction.value = prediction.apply(lambda row: row.value / 10, axis = 1)

In [151]:
team_df['value'] = team_df.apply(lambda row: prediction.value[prediction['name'] == row['name']].item(), axis = 1)

team_df['max_budget'] = team_df.apply(lambda row: row.value + budget, axis = 1)

In [154]:
prediction.head()

,name,GW,season,value,predicted_points,position
0,aaron connolly,38,21,5.2,0.165342,FWD
1,ademipo odubeko,38,21,4.5,0.120287,FWD
2,albian ajeti,38,21,5.5,0.120287,FWD
3,aleksandar mitrović,38,21,5.5,0.643287,FWD
4,alexandre lacazette,38,21,8.4,1.506591,FWD


In [152]:
team_df

,name,position,predicted_points,value,max_budget
0,harry kane,FWD,4.431238,11.9,16.1
1,neal maupay,FWD,0.665884,5.8,10.0
2,heung-min son,MID,4.290511,9.6,13.8
3,adam lallana,MID,0.993300,6.2,10.4
4,stuart dallas,DEF,4.681366,5.5,9.7
5,matt targett,DEF,2.859214,5.0,9.2
6,hugo lloris,GK,2.600000,5.6,9.8


In [91]:
assert (team_df['name'].isin(prediction['name'])).all(), "Not all team players in prediction dataset"

In [133]:
drop_duplicates = prediction[prediction['name'].duplicated()].index
prediction.drop(index = drop_duplicates, inplace = True)

In [176]:
# TODO: sort prediction by predicted score
sorted_prediction = prediction.sort_values(by = 'predicted_points', ascending = False)
drop_team = sorted_prediction[sorted_prediction['name'].isin(team_df['name'].values)].index
sorted_prediction.drop(index = drop_team, inplace = True)
sorted_prediction.reset_index(drop=True, inplace = True)

In [177]:
sorted_prediction

,name,GW,season,value,predicted_points,position
0,edinson cavani,38,21,8.0,7.075790,FWD
1,mohamed salah,38,21,12.9,6.516900,MID
2,vicente guaita,38,21,4.8,5.400000,GK
3,james ward-prowse,38,21,5.9,5.235958,MID
4,aaron ramsdale,38,21,4.6,5.000000,GK
...,...,...,...,...,...,...
690,niels nkounkou,38,21,4.0,-0.021618,DEF
691,jonjoe kenny,38,21,4.3,-0.035871,DEF
692,reece welch,38,21,4.0,-0.035871,DEF
693,kyle john,38,21,4.0,-0.035871,DEF


In [204]:
# TODO: calculate best affordable transfer for each player
team_df['incoming_player'] = 'no transfer'
team_df['incoming_points'] = 0
team_df['remaining_budget'] = team_df.max_budget

for team_index, team_row in team_df.iterrows():
    for predict_index, predict_row in sorted_prediction.iterrows():
        if team_row.position == predict_row.position\
            and team_row.predicted_points < predict_row.predicted_points\
            and team_row.max_budget > predict_row.value:
            team_df.loc[team_index, 'incoming_player'] = predict_row['name']
            team_df.loc[team_index, 'incoming_points'] = predict_row.predicted_points
            team_df.loc[team_index, 'remaining_budget'] = team_row.max_budget - predict_row.value
            break

team_df['points_difference'] = team_df.incoming_points - team_df.predicted_points



In [205]:
team_df

,name,position,predicted_points,value,max_budget,incoming,incoming_points,remaining_budget,points_difference,incoming_player
0,harry kane,FWD,4.431238,11.9,16.1,no transfer,7.075790,8.1,2.644551,edinson cavani
1,neal maupay,FWD,0.665884,5.8,10.0,no transfer,7.075790,2.0,6.409905,edinson cavani
2,heung-min son,MID,4.290511,9.6,13.8,no transfer,6.516900,0.9,2.226388,mohamed salah
3,adam lallana,MID,0.993300,6.2,10.4,no transfer,5.235958,4.5,4.242659,james ward-prowse
4,stuart dallas,DEF,4.681366,5.5,9.7,no transfer,0.000000,9.7,-4.681366,no transfer
5,matt targett,DEF,2.859214,5.0,9.2,no transfer,4.234607,3.5,1.375394,aaron cresswell
6,hugo lloris,GK,2.600000,5.6,9.8,no transfer,5.400000,5.0,2.800000,vicente guaita


In [206]:
sorted_prediction[sorted_prediction['name'].isin(team_df['incoming'])]

,name,GW,season,value,predicted_points,position


In [207]:
# TODO: add top 3 transfers to a list with outgoing player, incoming player & potential points increase
best_transfers = team_df.sort_values(by = 'points_difference', ascending = False)[['name', 'incoming_player', 'position', 'points_difference', 'remaining_budget']].head(3).reset_index(drop=True)


In [208]:
team_df

,name,position,predicted_points,value,max_budget,incoming,incoming_points,remaining_budget,points_difference,incoming_player
0,harry kane,FWD,4.431238,11.9,16.1,no transfer,7.075790,8.1,2.644551,edinson cavani
1,neal maupay,FWD,0.665884,5.8,10.0,no transfer,7.075790,2.0,6.409905,edinson cavani
2,heung-min son,MID,4.290511,9.6,13.8,no transfer,6.516900,0.9,2.226388,mohamed salah
3,adam lallana,MID,0.993300,6.2,10.4,no transfer,5.235958,4.5,4.242659,james ward-prowse
4,stuart dallas,DEF,4.681366,5.5,9.7,no transfer,0.000000,9.7,-4.681366,no transfer
5,matt targett,DEF,2.859214,5.0,9.2,no transfer,4.234607,3.5,1.375394,aaron cresswell
6,hugo lloris,GK,2.600000,5.6,9.8,no transfer,5.400000,5.0,2.800000,vicente guaita


In [209]:
sorted_prediction[sorted_prediction['position']=='GK']

,name,GW,season,value,predicted_points,position
2,vicente guaita,38,21,4.8,5.4,GK
4,aaron ramsdale,38,21,4.6,5.0,GK
5,bernd leno,38,21,5.0,5.0,GK
6,john ruddy,38,21,4.3,5.0,GK
7,kasper schmeichel,38,21,5.4,5.0,GK
...,...,...,...,...,...,...
680,joe hart,38,21,4.2,0.0,GK
681,andreas söndergaard,38,21,4.0,0.0,GK
682,adrián san miguel del castillo,38,21,4.2,0.0,GK
683,karl darlow,38,21,4.9,0.0,GK


In [210]:
best_transfers = best_transfers.rename(columns ={'name':'leaving_player'})

In [211]:
best_transfers

,leaving_player,incoming_player,position,points_difference,remaining_budget
0,neal maupay,edinson cavani,FWD,6.409905,2.0
1,adam lallana,james ward-prowse,MID,4.242659,4.5
2,hugo lloris,vicente guaita,GK,2.800000,5.0
